In [1]:
import metmhn.regularized_optimization as reg_opt
import metmhn.Utilityfunctions as utils

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', 
                      category=pd.errors.PerformanceWarning)

import jax.numpy as jnp
import numpy as np
import jax as jax
jax.config.update("jax_enable_x64", True)

import logging
# Adapt path to where logs should be kept
logging.basicConfig(filename='../logs/paad.log',
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    filemode='w', 
                    level=logging.INFO, 
                    force=True,
                    datefmt='%Y-%m-%d %H:%M:%S'
                    )

In the following example, we have information about the genotypes of Primary Tumors (PTs) and Metastases (MTs) for patients suffering from lung adenocarcinomas (LUADs). \
For some patients we only know the status of either the PT or the MT. This is indicated in our data in the column 'isPaired'. \
Additionally, the column 'metaStatus' indicates wether a metastasis was observed or not ('present', 'isMetastasis', 'absent'). \
This dataset contains mutations as well as copy number alterations. For this example we restrict this dataset to only contain mutations, such that it runs on average desktops on cpus only in ~20 minutes.

In [2]:
mut_handle = "../data/luad/G14_LUAD_Events.csv"
annot_handle = "../data/luad/G14_LUAD_sampleSelection.csv"
annot_data = pd.read_csv(annot_handle)
mut_data = pd.read_csv(mut_handle)
mut_data.rename(columns={"Unnamed: 0":"patientID"}, inplace = True)
dat = pd.merge(mut_data, annot_data.loc[:, ['patientID', 'metaStatus']], 
               on=["patientID", "patientID"])
muts = ['P.TP53 (M)', 'M.TP53 (M)', 'P.KRAS (M)', 'M.KRAS (M)', 'P.EGFR (M)', 'M.EGFR (M)', 'P.STK11 (M)', 'M.STK11 (M)', 'P.KEAP1 (M)', 'M.KEAP1 (M)', 
        'P.RBM10 (M)', 'M.RBM10 (M)', 'P.SMARCA4 (M)', 'M.SMARCA4 (M)', 'P.ATM (M)', 'M.ATM (M)', 'P.NF1 (M)', 'M.NF1 (M)', 'P.PTPRD (M)', 'M.PTPRD (M)', 
        'P.PTPRT (M)', 'M.PTPRT (M)', 'P.ARID1A (M)', 'M.ARID1A (M)', 'P.BRAF (M)', 'M.BRAF (M)', 'P.PIK3CA (M)', 'M.PIK3CA (M)', 'P.EPHA3 (M)', 'M.EPHA3 (M)', 
        'P.FAT1 (M)', 'M.FAT1 (M)', 'P.SETD2 (M)', 'M.SETD2 (M)', 'P.RB1 (M)', 'M.RB1 (M)', 'P.MET (M)', 'M.MET (M)', 'P.KMT2C (M)', 'M.KMT2C (M)']


Next, we clean the dataset and split it into 4 smaller datasets that contain only the never metastatsizing primary tumors, the metastasized unpaired primary tumors, the unpaired metastases and the paired data respectively.

In [5]:
# Label each datapoint with a numeric value according to its sequencetype
dat["type"] = dat.apply(utils.categorize, axis=1)
dat["Seeding"] = dat.apply(utils.add_seeding, axis=1)
dat.loc[dat["M.AgeAtSeqRep"] == "No metastasis included", "M.AgeAtSeqRep"] = pd.NA
dat.loc[dat["P.AgeAtSeqRep"] == "No primary included", "P.AgeAtSeqRep"] = pd.NA
dat["P.AgeAtSeqRep"] = dat["P.AgeAtSeqRep"].astype(pd.Int64Dtype())
dat["M.AgeAtSeqRep"] = dat["M.AgeAtSeqRep"].astype(pd.Int64Dtype())
dat["diag_order"] = dat["M.AgeAtSeqRep"] - dat["P.AgeAtSeqRep"]
dat["diag_order"] = dat["diag_order"].apply(lambda x: pd.NA if pd.isna(x) else 2 if x < 0 else 1 if x > 0 else 0) 
dat["diag_order"] = dat["diag_order"].astype(pd.Int64Dtype())
events_data = muts+["Seeding"]
# Only use datapoints where the state of the metastasis is known
cleaned = dat.loc[dat["type"].isin([0,1,2,3]), muts+["Seeding", "diag_order", "type"]]
# Remove completely empty datapoints
cleaned.drop(cleaned[cleaned.iloc[:,:-2].sum(axis=1)<1].index, inplace=True)
dat = jnp.array(cleaned.to_numpy(dtype=np.int8, na_value=-99))

Retrieve the event names and trim the PT/MT identifier:

In [6]:
events_plot = []
for elem in cleaned.columns[:-3].to_list()[::2]:
    full_mut_id = elem.split(".")
    events_plot.append(full_mut_id[1])
events_plot.append("Seeding")

Enumerate the frequencies of SNVs and CNVs in all subgroups. 'NM/EM' refer to Never Metastasizing/ Ever Metastasizing tumors, where only a single genotype is known. A Mutation is referred to as 'MT/PT-private' if happens exclusively in the MT or PT, otherwise it is called 'shared':

In [8]:
n_tot = (cleaned.shape[1]-1)//2 + 1
n_mut = n_tot-1
utils.marg_frequs(dat, events_plot)

Coupled (453)                   NM (547) EM-PT (1677) EM-MT (2127)
               PT-Private MT-Private Shared  Present      Present      Present
TP53 (M)             0.05       0.08   0.46     0.31         0.45         0.56
KRAS (M)             0.04       0.04   0.18     0.42         0.36         0.29
EGFR (M)             0.02       0.02   0.38     0.33         0.25         0.28
STK11 (M)            0.02       0.03   0.10     0.14         0.17         0.14
KEAP1 (M)            0.03       0.03   0.07     0.07         0.13         0.14
RBM10 (M)            0.02       0.02   0.06     0.16         0.11         0.10
SMARCA4 (M)          0.01       0.03   0.03     0.03         0.08         0.08
ATM (M)              0.02       0.03   0.04     0.05         0.06         0.07
NF1 (M)              0.02       0.03   0.03     0.06         0.06         0.07
PTPRD (M)            0.03       0.02   0.04     0.05         0.06         0.06
PTPRT (M)            0.02       0.02   0.03     0.04         0.05         0.05
ARID1A (M)           0.02       0.05   0.03     0.03         0.04         0.06
BRAF (M)             0.02       0.01   0.03     0.05         0.06         0.05
PIK3CA (M)           0.02       0.03   0.04     0.06         0.05         0.04
EPHA3 (M)            0.02       0.01   0.04     0.05         0.05         0.05
FAT1 (M)             0.02       0.02   0.02     0.04         0.05         0.05
SETD2 (M)            0.02       0.01   0.02     0.05         0.04         0.05
RB1 (M)              0.02       0.01   0.03     0.02         0.04         0.05
MET (M)              0.01       0.01   0.03     0.05         0.05         0.04
KMT2C (M)            0.01       0.02   0.02     0.03         0.04         0.05
Seeding              1.00       0.00   0.00     0.00         1.00       -99.00

Optional: We use a  sparsity and symmetry promoting penalty. The weight of the penalization can be determined in a k-fold crossvalidation:

In [ ]:
log_lams = np.linspace(-4, -2, 5)
lams = 10**log_lams
utils.cross_val(cleaned.copy(), events_data, lams, 5, 0.83)

Train an MHN on the full dataset:

In [9]:
penal = 0.0005
m_p_corr = 0.65
th_init, dp_init, dm_init = utils.indep(dat[:,:-2], cleaned.shape[0])
theta, d_p, d_m= reg_opt.learn_mhn(th_init, dp_init, dm_init, dat,m_p_corr, reg_opt.symmetric_penal, penal)

AttributeError: 'numpy.ndarray' object has no attribute 'at'

In [ ]:
th_plot = np.row_stack((d_p.reshape((1,-1)), 
                        d_m.reshape((1,-1)), 
                        theta))

Visualize the results:

In [ ]:
utils.plot_theta(th_plot, events_plot, 0.2)

In [ ]:
df2 = pd.DataFrame(th_plot, columns=events_plot)
df2.to_csv("../results/luad/luad_g14_19muts.csv")
